# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
from logger import get_logger
_logs = get_logger(__name__)


In [2]:
import dask.dataframe as dd

c:\Users\Hossein-Bg\anaconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\Hossein-Bg\AppData\Local\Temp\ipykernel_11172\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedba

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
# Load environment variable for PRICE_DATA
price_data_dir = os.getenv('PRICE_DATA')
price_data_dir


'../../05_src/data/prices/'

In [4]:
# Use glob to find all parquet files in the directory
parquet_files = glob(os.path.join(price_data_dir,"*/*/part.0.parquet"))
# parquet_files

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

c:\Users\Hossein-Bg\anaconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.
dd_feat = (dd_px.groupby('ticker', group_keys=False)
           .apply(
               lambda x: x.assign(
                   Close_lag_1 = x['Close'].shift(1),
                   Adj_Close_lag_1 = x['Adj Close'].shift(1)
               )
           )
           .assign(
               returns = lambda x: x['Adj Close'] / x['Adj_Close_lag_1'] - 1,
               hi_lo_range = lambda x: x['High'] - x['Low']
           )
)

print(dd_feat.head())

C:\Users\Hossein-Bg\AppData\Local\Temp\ipykernel_11172\211774174.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False)
c:\Users\Hossein-Bg\anaconda3\envs\dsi_participant\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


             Date    Open   High     Low   Close  Adj Close   Volume  \
ticker                                                                 
HUM    2000-01-03  8.3125  8.375  7.3750  7.5625   6.771275  1287300   
HUM    2000-01-04  7.3750  7.875  7.3750  7.6250   6.827235  1238300   
HUM    2000-01-05  7.5000  7.875  7.5000  7.8125   6.995120  1096300   
HUM    2000-01-06  7.7500  8.250  7.5000  8.1250   7.274925  1026700   
HUM    2000-01-07  8.1875  9.125  8.0625  8.7500   7.834534  2419300   

             sector            subsector  year  Close_lag_1  Adj_Close_lag_1  \
ticker                                                                         
HUM     Health Care  Managed Health Care  2000          NaN              NaN   
HUM     Health Care  Managed Health Care  2000       7.5625         6.771275   
HUM     Health Care  Managed Health Care  2000       7.6250         6.827235   
HUM     Health Care  Managed Health Care  2000       7.8125         6.995120   
HUM     Health 

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [6]:
# Write your code below.
# Convert the Dask data frame to a pandas data frame
pd_feat = dd_feat.compute()

# Reset the index to ensure there are no duplicate labels during the groupby operation
pd_feat = pd_feat.reset_index()

# Add a rolling average return calculation with a window of 10 days, grouped by 'ticker'
pd_feat['rolling_avg_return_10d'] = (
    pd_feat.groupby('ticker')['returns']
    .rolling(window=10, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)  # Drop the groupby index to align with the main DataFrame index
)

print(pd_feat.head())

c:\Users\Hossein-Bg\anaconda3\envs\dsi_participant\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


  ticker       Date    Open   High     Low   Close  Adj Close   Volume  \
0    HUM 2000-01-03  8.3125  8.375  7.3750  7.5625   6.771275  1287300   
1    HUM 2000-01-04  7.3750  7.875  7.3750  7.6250   6.827235  1238300   
2    HUM 2000-01-05  7.5000  7.875  7.5000  7.8125   6.995120  1096300   
3    HUM 2000-01-06  7.7500  8.250  7.5000  8.1250   7.274925  1026700   
4    HUM 2000-01-07  8.1875  9.125  8.0625  8.7500   7.834534  2419300   

        sector            subsector  year  Close_lag_1  Adj_Close_lag_1  \
0  Health Care  Managed Health Care  2000          NaN              NaN   
1  Health Care  Managed Health Care  2000       7.5625         6.771275   
2  Health Care  Managed Health Care  2000       7.6250         6.827235   
3  Health Care  Managed Health Care  2000       7.8125         6.995120   
4  Health Care  Managed Health Care  2000       8.1250         7.274925   

    returns  hi_lo_range  rolling_avg_return_10d  
0       NaN       1.0000                     NaN  
1 

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Was it necessary to convert to pandas to calculate the moving average return?

It was not strictly necessary to convert the Dask DataFrame to a Pandas DataFrame just to calculate the moving average return. Dask supports the .rolling() method, so you could have calculated the rolling average directly within the Dask DataFrame without converting it to Pandas.

Would it have been better to do it in Dask? Why?

Yes, it would generally be better to do this in Dask if you are dealing with a large dataset that may not fit into memory. The main reason to stay in Dask for this calculation is its ability to handle larger-than-memory data by parallelizing operations and using lazy evaluation.
By computing the rolling average directly in Dask, you avoid potential memory constraints when working with very large datasets. Converting to Pandas (.compute()) forces the entire DataFrame into memory, which can be problematic if it exceeds available RAM.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.